In [1]:
import numpy as np
import os
import json
from PIL import Image
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.applications import ResNet50, VGG19
from keras import activations, losses, optimizers
import numpy as np
import shapely.wkt
import shapely
from shapely.geometry import Polygon


from tqdm import tqdm

Using TensorFlow backend.
/home/abhirag/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/abhirag/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/abhirag/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/abhirag/.local/lib/python3.6/site-packages/tensorflow/py

In [2]:
vgg_model = VGG19()

In [3]:
vgg_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
## taking bottleneck features from VGG19
vgg_model.trainable = False
my_layer = Flatten()(vgg_model.layers[11].output)
model = Model(inputs = vgg_model.input, outputs = my_layer)

In [6]:
model.output

<tf.Tensor 'flatten_1/Reshape:0' shape=(?, ?) dtype=float32>

In [7]:
model.predict(np.random.randn(1, 224, 224, 3)).shape

(1, 200704)

In [5]:
def process_img(img_array, polygon_pts):
    height, width, _ = img_array.shape

    xcoords = polygon_pts[:, 0]
    ycoords = polygon_pts[:, 1]
    xmin, xmax = np.min(xcoords), np.max(xcoords)
    ymin, ymax = np.min(ycoords), np.max(ycoords)

    xdiff = xmax - xmin
    ydiff = ymax - ymin

    return img_array[int(ymin):int(ymax), int(xmin):int(xmax), :]

In [6]:
base = '/home/abhirag/ONLY_PORTUGAL/'

images_dir = os.path.join(base, 'images')
jsons_dir = os.path.join(base, 'labels')

jsons_list = [os.path.join(jsons_dir, i) for i in os.listdir(jsons_dir)]

len(jsons_list)

3000

In [10]:
jsons_list[0]

'/home/abhirag/ONLY_PORTUGAL/labels/portugal-wildfire_00001642_post_disaster.json'

In [11]:
## getiing the patch ids 
images_uids = {}


for json_file in jsons_list:
    if json_file.split('_')[3] == 'post':
        with open(json_file) as f:
            data = json.load(f)
        images_uids[data['metadata']['img_name']] = []
        for j in range(len(data['features']['xy'])):
            polygon_id = data['features']['xy'][j]['properties']['uid']
            images_uids[data['metadata']['img_name']].append(polygon_id)

print(len(images_uids))

1500


In [12]:
images_uids['portugal-wildfire_00001642_post_disaster.png']

['e0989c1b-6ea8-46ce-b5ff-c954d0e94a1d',
 '27c75198-f5bb-4e78-be95-ad52088e54f1',
 '6f6b73a7-7886-4e65-bc86-3e8853f5eb68',
 '6a923323-1d67-4829-8dd8-fde3a81cebd2']

In [13]:
### getting a dictionary with pre, post image data with corresponding labels
training_data = {}

for key in tqdm(list(images_uids.keys())):
    
    # key = 'portugal-wildfire_00001642_post_disaster.png'

    training_data[key] = {}

    post_json = json.load(open('/home/abhirag/ONLY_PORTUGAL/labels/' + key.replace('png', 'json')))
    pre_json = json.load(open('/home/abhirag/ONLY_PORTUGAL/labels/' + key.replace('png', 'json').replace('post', 'pre')))

    pre_image = np.array(Image.open('/home/abhirag/ONLY_PORTUGAL/images/' + key.replace('post', 'pre')))
    post_image = np.array(Image.open('/home/abhirag/ONLY_PORTUGAL/images/' + key))

    for polygon_id in images_uids[key]:

        training_data[key][polygon_id] = {}

        for x,y in zip(range(len(post_json['features']['xy'])), range(len(pre_json['features']['xy']))):

            if post_json['features']['xy'][x]['properties']['uid'] == polygon_id:
                poly = np.array(list(shapely.wkt.loads(post_json['features']['xy'][x]['wkt']).exterior.coords))
                training_data[key][polygon_id]['post'] = process_img(post_image, poly)
                training_data[key][polygon_id]['label'] = post_json['features']['xy'][x]['properties']['subtype']

            if pre_json['features']['xy'][y]['properties']['uid'] == polygon_id:
                poly = np.array(list(shapely.wkt.loads(pre_json['features']['xy'][y]['wkt']).exterior.coords))
                training_data[key][polygon_id]['pre'] = process_img(pre_image, poly)



100%|██████████| 1500/1500 [01:29<00:00, 16.77it/s]


In [22]:
import pickle
with open('/home/abhirag/experiments_on_portugal/training_data_VGG_3rd_pooling_layer.pkl', 'wb') as fp:
    pickle.dump(training_data, fp)

# MODEL

In [23]:
training_data['portugal-wildfire_00001642_post_disaster.png']['e0989c1b-6ea8-46ce-b5ff-c954d0e94a1d']['pre'].shape

(37, 68, 3)

In [24]:
### just cheking and confirming the shapes of pre and post patches is same
for i in list(training_data.keys()):
    for j in list(training_data[i].keys()):
        print(training_data[i][j]['pre'].shape)
        print(training_data[i][j]['post'].shape)
        print(training_data[i][j]['label'])
        
#         break
    break

(37, 68, 3)
(37, 68, 3)
no-damage
(37, 41, 3)
(37, 41, 3)
no-damage
(15, 21, 3)
(15, 21, 3)
no-damage
(10, 9, 3)
(10, 9, 3)
un-classified


In [25]:
i = 'portugal-wildfire_00001642_post_disaster.png'
j = 'e0989c1b-6ea8-46ce-b5ff-c954d0e94a1d'

pre_img = np.array(Image.fromarray(training_data[i][j]['pre']).resize((224, 224)))
post_img = np.array(Image.fromarray(training_data[i][j]['post']).resize((224, 224)))


x = model.predict(pre_img.reshape(-1, 224, 224, 3))
y = model.predict(post_img.reshape(-1, 224, 224, 3))

target = training_data[i][j]['label']

In [30]:
model.predict(np.random.randn(1, 224, 224, 3))

array([[29.717209, 94.79503 , 46.748394, ..., 33.656433,  0.      ,
         8.166745]], dtype=float32)

In [31]:
### getting the bottolneck features for pre and post images and multiplying the two vectors and storing it along with lables
from sklearn.model_selection import train_test_split
from tqdm import tqdm
def preprocess_data(training_data, train_size=0.7):
    xdata, ydata = [], []
    X1_data, X2_data = [], []
    for img_name in tqdm(list(training_data.keys())):
        for uid in list(training_data[img_name].keys()):
            pre_img = np.array(Image.fromarray(training_data[img_name][uid]['pre']).resize((224, 224)))
            post_img = np.array(Image.fromarray(training_data[img_name][uid]['post']).resize((224, 224)))
            
            X1 = model.predict(pre_img.reshape(-1, 224, 224, 3))
            X2 = model.predict(post_img.reshape(-1, 224, 224, 3))
        
            X1_data.append(X1)
            X2_data.append(X2)
            xdata.append(np.multiply(X1, X2) / (np.linalg.norm(X1, 2) * np.linalg.norm(X2, 2)))
            ydata.append(training_data[img_name][uid]['label'])
    X, y = np.array(xdata), np.array(ydata)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size)
    return Xtrain, Xtest, ytrain, ytest, X1_data, X2_data



In [33]:
Xtrain, Xtest, ytrain, ytest, X1_data, X2_data = preprocess_data(training_data)

100%|██████████| 1500/1500 [38:08<00:00,  1.53s/it] 


In [34]:
Xtrain.shape, Xtest.shape

((13565, 1, 200704), (5814, 1, 200704))

In [49]:
np.save('vgg_layer11_dumps/xtrain', Xtrain)
np.save('vgg_layer11_dumps/xtest', Xtest)
np.save('vgg_layer11_dumps/ytrain', ytrain)
np.save('vgg_layer11_dumps/ytest', ytest)
np.save('vgg_layer11_dumps/pre_image_bottlecks', X1_data)
np.save('vgg_layer11_dumps/post_image_bottlecks', X2_data)


In [7]:
Xtrain, Xtest, ytrain, ytest = np.load('vgg_layer11_dumps/xtrain.npy'), np.load('vgg_layer11_dumps/xtest.npy'), np.load('vgg_layer11_dumps/ytrain.npy'), np.load('vgg_layer11_dumps/ytest.npy')

In [8]:
Xtrain.shape

(13565, 1, 200704)

In [9]:
ytrain.shape

(13565,)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier

/home/abhirag/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [11]:
indices_to_drop = np.where(ytrain == 'un-classified')[0]

Xtrain_new = np.delete(Xtrain, indices_to_drop, axis = 0)
ytrain_new = np.delete(ytrain, indices_to_drop)

Xtrain_new.shape, ytrain_new.shape

((12975, 1, 200704), (12975,))

In [13]:
indices_to_drop_2 = np.where(ytest == 'un-classified')[0]

Xtest_new = np.delete(Xtest, indices_to_drop_2, axis = 0)
ytest_new = np.delete(ytest, indices_to_drop_2)

Xtest_new.shape, ytest_new.shape

((5559, 1, 200704), (5559,))

In [14]:
indices_no_damage = np.where(ytrain == 'no-damage')[0]

# indices_no_damage

np.random.shuffle(indices_no_damage)


Xtrain_new_balanced = np.delete(Xtrain, indices_no_damage[:-300], axis = 0)
ytrain_new_balanced = np.delete(ytrain, indices_no_damage[:-300])


Xtrain_new_balanced.shape, ytrain_new_balanced.shape

((1803, 1, 200704), (1803,))

In [54]:
rfc = RandomForestClassifier(n_estimators = 50, n_jobs = -1, class_weight = 'balanced')
rfc.fit(Xtrain_new_balanced.reshape(1803, 200704), ytrain_new_balanced)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=50, n_jobs=-1, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [55]:
print(classification_report(y_pred = rfc.predict(Xtest_new.reshape(5559, 200704)), y_true = ytest_new))

               precision    recall  f1-score   support

    destroyed       0.09      0.84      0.16       267
 major-damage       0.00      0.00      0.00        68
 minor-damage       0.00      0.00      0.00        54
    no-damage       0.99      0.34      0.51      5170
un-classified       0.00      0.00      0.00         0

     accuracy                           0.36      5559
    macro avg       0.22      0.24      0.13      5559
 weighted avg       0.93      0.36      0.48      5559



/home/abhirag/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/abhirag/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

              precision    recall  f1-score   support

   destroyed       0.05      1.00      0.09       267
major-damage       0.00      0.00      0.00        68
minor-damage       0.00      0.00      0.00        54
   no-damage       0.00      0.00      0.00      5170

    accuracy                           0.05      5559
   macro avg       0.01      0.25      0.02      5559
weighted avg       0.00      0.05      0.00      5559



/home/abhirag/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
xgbc = XGBClassifier(silent=False)
xgbc.fit(Xtrain_new_balanced.reshape(1803, 200704), ytrain_new_balanced)

KeyboardInterrupt: 

In [ ]:
print(classification_report(y_pred = xgbc.predict(Xtest_new.reshape(5569, 4096)), y_true = ytest_new))